In [1]:
import json
import sqlite3
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

In [2]:
con = sqlite3.connect('../data/interim/articles_with_author_mapping.db')
cur = con.cursor()

In [22]:
cur.execute("select id, name, abbreviation, matching_certainty from authors")
rows = cur.fetchall()
authors = pd.DataFrame(columns=["id", "name", "abbreviation", "certainty"], data=rows)
authors.set_index("id", inplace=True)

In [23]:
# convert null to nans
authors = authors.replace("null", np.nan)

,name,abbreviation,certainty
id,,,
1,kai-uwe brandt,kub,0.800
2,jan peter,jap,0.600
3,manfred lüttich,maf,0.600
4,heiko trebs,ht,0.800
5,andreas tappert,art,0.672
...,...,...,...
63618,von Winfried Mahr,NaN,NaN
63619,wertvollen Erfahrungen,NaN,NaN
63620,die sie prägen. Frank Pfeifer,NaN,NaN


In [26]:
# set certainty of a row to the average certainty over all rows with same name and abbreviation, ignore nan values
mean_certainty = authors.groupby(["name", "abbreviation"])["certainty"].transform(lambda x: np.nanmean(x))
authors["certainty"] = mean_certainty
authors

/tmp/ipykernel_29365/1272401548.py:2: RuntimeWarning: Mean of empty slice
  mean_certainty = authors.groupby(["name", "abbreviation"])["certainty"].transform(lambda x: np.nanmean(x))


,name,abbreviation,certainty
id,,,
1,kai-uwe brandt,kub,0.810345
2,jan peter,jap,0.600000
3,manfred lüttich,maf,0.600000
4,heiko trebs,ht,0.800000
5,andreas tappert,art,0.685579
...,...,...,...
63618,von Winfried Mahr,NaN,NaN
63619,wertvollen Erfahrungen,NaN,NaN
63620,die sie prägen. Frank Pfeifer,NaN,NaN


In [33]:
# set the number of distinct name, abbreviation rows
authors["name_abbreviation_count"] = authors.groupby(["name", "abbreviation"])["name"].transform("count")

In [37]:
# drop duplicates based on name, abbreviation, certainty and name_abbreviation_count
authors.drop_duplicates(subset=["name", "abbreviation", "certainty", "name_abbreviation_count"], inplace=True)

In [42]:
# normalize name_abbreviation_count for each name
authors["name_abbreviation_count"] = authors.groupby("name")["name_abbreviation_count"].transform(lambda x: (x- x.min()) / (x.max() - x.min()))

In [ ]:
# todo 09.07 den count richtig normalizen, damit die calculation schneller geht udn dann den score aus der anderen richtung berechnen

In [41]:
# Generate edge weights based on certainty and frequency

def mapping_func(abbr):
    abbr = calc_abbreviation_to_name_score(abbr)
    return abbr

def name_mapping_func(name):
    print("Name: " + name)
    # get all names pointing to that abbreviation
    abbreviations = authors[authors.name == name].abbreviation
    certainties = authors[authors.name == name][["abbreviation", "certainty"]].groupby("abbreviation").agg({"certainty": "mean"}).reset_index()
    frequencies = abbreviations.value_counts(normalize=True)
    # merge certainties and frequencies on name
    merged = pd.merge(certainties, frequencies, on="abbreviation")
    print(merged)
    return name

def calc_abbreviation_to_name_score(row):
    abbr, name = row["abbreviation"], row["name"]
    # get all names pointing to that abbreviation
    names = authors[authors.abbreviation == abbr].name
    # get the proportion of the name in the list
    proportions = names.value_counts(normalize=True).to_frame().reset_index()
    proportion = proportions[proportions["name"] == name]["proportion"].iloc[0]
    return proportion

# loop through names
authors["abbreviation_to_name_score"] = authors.apply(lambda x: calc_abbreviation_to_name_score(x), axis=1)
authors

IndexError: single positional indexer is out-of-bounds